In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_ML1M'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-1m'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sklearn as sklearn
import datetime as dt 
import seaborn as sns
import torch as torch
from sklearn import metrics
import pickle

In [4]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df.pkl','rb'))

In [5]:
dftrain.shape, dfvalid.shape

((895738, 18), (104471, 18))

In [6]:
dftrain.columns

Index(['age', 'flag_train', 'gender', 'genre', 'itemId', 'maxSeqNum', 'month',
       'month_acq', 'occupation', 'rating', 'seqNum', 'timestamp', 'title',
       'userId', 'year', 'year_acq', 'year_month_acq', 'zip'],
      dtype='object')

In [7]:
item_features=['genre','title']
user_features=['gender','occupation','zip']

In [8]:
dftrain[item_features].head(),dftrain[user_features].head()

(                             genre  \
 0                           Comedy   
 1                Children's|Sci-Fi   
 2                     Comedy|Drama   
 3  Action|Adventure|Fantasy|Sci-Fi   
 4                       Comedy|War   
 
                                               title  
 0                    Stuart Saves His Family (1995)  
 1                 Return from Witch Mountain (1978)  
 2     Indian Summer (a.k.a. Alive & Kicking) (1996)  
 3  Star Wars: Episode I - The Phantom Menace (1999)  
 4                            In the Army Now (1994)  ,
   gender  occupation    zip
 0      M          12  30303
 1      M          12  30303
 2      M          12  30303
 3      M          12  30303
 4      M          12  30303)

In [9]:
dftrain['genres']=[item.replace("|", " ") for item in dftrain['genre']]
dfvalid['genres']=[item.replace("|", " ") for item in dfvalid['genre']]

In [10]:
from sklearn import feature_extraction, preprocessing
myvec_genre=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True)
one_hot_genre_train=myvec_genre.fit_transform(dftrain['genres'])
one_hot_genre_valid=myvec_genre.transform(dfvalid['genres'])

In [11]:
from sklearn import feature_extraction, preprocessing
myvec_title=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True,max_features=250)
one_hot_title_train=myvec_title.fit_transform(dftrain['title'])
one_hot_title_valid=myvec_title.transform(dfvalid['title'])

In [12]:
from sklearn import feature_extraction, preprocessing
myvec_tag=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True,max_features=100)
one_hot_tag_train=myvec_tag.fit_transform(dftrain['title'])
one_hot_tag_valid=myvec_tag.transform(dfvalid['title'])

In [13]:
one_hot_title_train.shape, one_hot_genre_train.shape,one_hot_tag_train.shape

((895738, 250), (895738, 20), (895738, 100))

In [14]:
one_hot_title_train

<895738x250 sparse matrix of type '<class 'numpy.int64'>'
	with 2019829 stored elements in Compressed Sparse Row format>

In [15]:
df_sideinfo_title_train=pd.DataFrame(one_hot_title_train.todense(),columns=['f_title_'+x for x in myvec_title.get_feature_names()])
df_sideinfo_genre_train=pd.DataFrame(one_hot_genre_train.todense(),columns=['f_genre_'+x for x in myvec_genre.get_feature_names()])
df_sideinfo_tag_train=pd.DataFrame(one_hot_tag_train.todense(),columns=['f_tag_'+x for x in myvec_tag.get_feature_names()])

In [16]:
df_sideinfo_title_valid=pd.DataFrame(one_hot_title_valid.todense(),columns=['f_title_'+x for x in myvec_title.get_feature_names()])
df_sideinfo_genre_valid=pd.DataFrame(one_hot_genre_valid.todense(),columns=['f_genre_'+x for x in myvec_genre.get_feature_names()])
df_sideinfo_tag_valid=pd.DataFrame(one_hot_tag_valid.todense(),columns=['f_tag_'+x for x in myvec_tag.get_feature_names()])

In [17]:
df_sideinfo_title_train.shape,df_sideinfo_genre_train.shape,df_sideinfo_tag_train.shape

((895738, 250), (895738, 20), (895738, 100))

In [18]:
df_sideinfo_title_valid.shape,df_sideinfo_genre_valid.shape,df_sideinfo_tag_valid.shape

((104471, 250), (104471, 20), (104471, 100))

In [19]:
dftrain.shape

(895738, 19)

In [20]:
dftrain.dtypes

age                        int64
flag_train                 int64
gender                    object
genre                     object
itemId                     int64
maxSeqNum                float64
month                      int64
month_acq                  int64
occupation                 int64
rating                     int64
seqNum                     int64
timestamp         datetime64[ns]
title                     object
userId                     int64
year                       int64
year_acq                   int64
year_month_acq             int64
zip                       object
genres                    object
dtype: object

In [21]:
dftrain.shape,df_sideinfo_title_train.shape,df_sideinfo_genre_train.shape,df_sideinfo_tag_train.shape

((895738, 19), (895738, 250), (895738, 20), (895738, 100))

In [22]:
dftrain.reset_index(inplace=True)
dfvalid.reset_index(inplace=True)

In [23]:
dftrain=pd.concat([dftrain,df_sideinfo_title_train,df_sideinfo_genre_train,df_sideinfo_tag_train],axis=1)

In [24]:
dfvalid=pd.concat([dfvalid,df_sideinfo_title_valid,df_sideinfo_genre_valid,df_sideinfo_tag_valid],axis=1)

In [25]:
dftrain.shape,dfvalid.shape

((895738, 390), (104471, 390))

In [26]:
dftrain.dtypes

index                        int64
age                          int64
flag_train                   int64
gender                      object
genre                       object
itemId                       int64
maxSeqNum                  float64
month                        int64
month_acq                    int64
occupation                   int64
rating                       int64
seqNum                       int64
timestamp           datetime64[ns]
title                       object
userId                       int64
year                         int64
year_acq                     int64
year_month_acq               int64
zip                         object
genres                      object
f_title_1939                 int64
f_title_1940                 int64
f_title_1941                 int64
f_title_1942                 int64
f_title_1944                 int64
f_title_1946                 int64
f_title_1950                 int64
f_title_1951                 int64
f_title_1953        

In [27]:
DATAPATH

'/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-1m'

In [28]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df_side.pkl','wb'))